In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from top2vec import Top2Vec

import random
import pickle

from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from itertools import combinations
from gensim import corpora

import numpy as np
from octis.evaluation_metrics.diversity_metrics import TopicDiversity, InvertedRBO
import IPython.display as ipd

import gc
import time
import warnings
warnings.filterwarnings(action='ignore')

2025-05-10 13:35:37.751811: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-10 13:35:38.008918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746862538.114875   73500 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746862538.146316   73500 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-10 13:35:38.388392: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Data Prep

In [3]:
with open('./pickled/banfake_string.pickle', 'rb') as f:
    docs = pickle.load(f)

num_topics = 12

In [4]:
len(docs)

48678

In [5]:
random.shuffle(docs)

## Functions

In [6]:
def bengali_tokenizer(text):
    return text.split()

In [7]:
def bengali_sentencizer(text):
    return text.split('।')

In [8]:
def topic_modeling(docs, num_topics, embedding_model='doc2vec'):

    model = Top2Vec(
        documents=docs,
        embedding_model=embedding_model,
        gpu_hdbscan=True,
        gpu_umap=True,
        umap_args={'n_components':10,
                   'metric':'cosine'},
        tokenizer=bengali_tokenizer,
        sentencizer=bengali_sentencizer,
        ngram_vocab=False,
        use_embedding_model_tokenizer=False,
        workers=4,
        keep_documents=False,
        embedding_batch_size=512,
        verbose=True)

    doc_ids = list(range(len(docs)))
    topic_num = model.get_num_topics()  

    if topic_num > num_topics:
        print(f'{topic_num} topics are being reduced to {num_topics} topics.')
        model.hierarchical_topic_reduction(num_topics=num_topics)
        topic_words, _, _ = model.get_topics(reduced=True)

        
    elif topic_num == num_topics:
        topic_words, _, _ = model.get_topics()
        
    else: 
        print(f'{topic_num} topics identified.')
        topic_words, _, _ = model.get_topics()

    top10_topic_words = [words[:10] for words in topic_words]

    return top10_topic_words, topic_num

In [9]:
def evaluate_topic_words(topic_words, docs):

    tokenized_docs = [bengali_tokenizer(doc) for doc in docs]
    texts = [doc for doc in tokenized_docs if len(doc) > 0]
    
    dictionary = Dictionary(texts)

    results = {}
    
    if texts and dictionary:
        
        coherence_c_v = CoherenceModel(topics=topic_words, texts=texts, dictionary=dictionary, coherence='c_v').get_coherence()
        coherence_npmi = CoherenceModel(topics=topic_words, texts=texts, dictionary=dictionary, coherence='c_npmi').get_coherence()
        
        results['coherence_c_v'] = coherence_c_v
        results['coherence_npmi'] = coherence_npmi
    
    octis = {
        'topics': topic_words  
    }
    
    td = TopicDiversity(topk=10)
    irbo = InvertedRBO(topk=10)
    
    results['topic_diversity'] = td.score(octis)
    results['IRBO'] = irbo.score(octis)
    
    return results

In [10]:
start_time = time.time() 
topic_words, topic_num = topic_modeling(docs, 
                             num_topics=num_topics, 
                             embedding_model='doc2vec')
end_time = time.time() 
runtime = end_time - start_time

2025-05-10 13:35:47,294 - top2vec - INFO - Pre-processing documents for training
2025-05-10 13:35:48,868 - top2vec - INFO - Creating joint document/word embedding
2025-05-10 14:01:39,876 - top2vec - INFO - Creating lower dimension embedding of documents
2025-05-10 14:01:41,601 - top2vec - INFO - Finding dense areas of documents
2025-05-10 14:01:42,308 - top2vec - INFO - Finding topics


507 topics are being reduced to 12 topics.


Reducing topics: 100%|████████████████████████| 495/495 [01:16<00:00,  6.49it/s]


In [11]:
gc.collect()

233

In [12]:
for idx, words in enumerate(topic_words):
    print(f'{idx} : {list(words)}')

0 : ['প্রতিষ্ঠানের', 'খাতের', 'শিক্ষার্থীদের', 'চাকরির', 'গ্রাহকদের', 'শিক্ষার্থীরা', 'গ্রাহকরা', 'করপোরেট', 'সফটওয়্যারের', 'উদ্যোক্তা']
1 : ['ময়নাতদন্ত', 'ময়নাতদন্তের', 'মর্গে', 'সুরতহাল', 'স্থানীয়রা', 'ওসি', 'লোকজন', 'শিশুটির', 'ময়না', 'কর্তব্যরত']
2 : ['রয়টার্স', 'দেশটি', 'দেশটির', 'ট্রাম্প', 'ওয়াশিংটনের', 'মার্কিন', 'পশ্চিমা', 'যুক্তরাষ্ট্রকে', 'পার্সটুডে', 'ট্রাম্পের']
3 : ['ব্যাটিং', 'ব্যাটিংয়ের', 'ব্যাটসম্যানদের', 'ব্যাটসম্যান', 'মুশফিক', 'ব্যাটসম্যানরা', 'কাপে', 'ওভার', 'বোলিং', 'বোলার']
4 : ['বিএনপি', 'বিএনপিকে', 'বিএনপির', 'ঐক্যের', 'জনগণ', 'নেতাদের', 'ঐক্য', 'গণতন্ত্র', 'ঐক্যবদ্ধ', 'ফখরুল']
5 : ['উপজেলা', 'হাতকে', 'আলহাজ্ব', 'উন্নয়নমূলক', 'লীগের', 'আলহাজ', 'এমপি', 'আওয়ামী', 'ছাত্রনেতা', 'উন্নয়নের']
6 : ['উপকারী', 'অভ্যাস', 'খেতে', 'অসুখ', 'খাওয়া', 'চর্বি', 'খাওয়ার', 'উপকার', 'খাবারে', 'ব্যায়াম']
7 : ['চলাচলের', 'বাসিন্দাদের', 'পানির', 'জোয়ারের', 'পানি', 'সড়কটি', 'বর্ষার', 'ভোগান্তি', 'অনুপযোগী', 'বৃষ্টিপাত']
8 : ['সিনেমায়', 'ছবির', 'নায়িকা', 'অভিনয়', 'অভিন

In [13]:
topicz = [] 
for words in topic_words:
    topicz.append(list(words))

In [14]:
topicz

[['প্রতিষ্ঠানের',
  'খাতের',
  'শিক্ষার্থীদের',
  'চাকরির',
  'গ্রাহকদের',
  'শিক্ষার্থীরা',
  'গ্রাহকরা',
  'করপোরেট',
  'সফটওয়্যারের',
  'উদ্যোক্তা'],
 ['ময়নাতদন্ত',
  'ময়নাতদন্তের',
  'মর্গে',
  'সুরতহাল',
  'স্থানীয়রা',
  'ওসি',
  'লোকজন',
  'শিশুটির',
  'ময়না',
  'কর্তব্যরত'],
 ['রয়টার্স',
  'দেশটি',
  'দেশটির',
  'ট্রাম্প',
  'ওয়াশিংটনের',
  'মার্কিন',
  'পশ্চিমা',
  'যুক্তরাষ্ট্রকে',
  'পার্সটুডে',
  'ট্রাম্পের'],
 ['ব্যাটিং',
  'ব্যাটিংয়ের',
  'ব্যাটসম্যানদের',
  'ব্যাটসম্যান',
  'মুশফিক',
  'ব্যাটসম্যানরা',
  'কাপে',
  'ওভার',
  'বোলিং',
  'বোলার'],
 ['বিএনপি',
  'বিএনপিকে',
  'বিএনপির',
  'ঐক্যের',
  'জনগণ',
  'নেতাদের',
  'ঐক্য',
  'গণতন্ত্র',
  'ঐক্যবদ্ধ',
  'ফখরুল'],
 ['উপজেলা',
  'হাতকে',
  'আলহাজ্ব',
  'উন্নয়নমূলক',
  'লীগের',
  'আলহাজ',
  'এমপি',
  'আওয়ামী',
  'ছাত্রনেতা',
  'উন্নয়নের'],
 ['উপকারী',
  'অভ্যাস',
  'খেতে',
  'অসুখ',
  'খাওয়া',
  'চর্বি',
  'খাওয়ার',
  'উপকার',
  'খাবারে',
  'ব্যায়াম'],
 ['চলাচলের',
  'বাসিন্দাদের',
  'পানির',
  'জোয়ারের',
 

In [15]:
topic_num, runtime

(507, 1632.943116426468)

In [16]:
# results = evaluate_topic_words(topic_words, docs)
# results['runtime'] = runtime
# results['topics'] = topic_num
# results

In [17]:
gc.collect()

0

In [18]:
##########################################################################################################################################

In [19]:
# start_time = time.time() 
# topic_words, topic_num = topic_modeling(docs, 
#                              num_topics=num_topics, 
#                              embedding_model='paraphrase-multilingual-MiniLM-L12-v2')
# end_time = time.time() 
# runtime = end_time - start_time

In [20]:
# results = evaluate_topic_words(topic_words, docs)
# results['runtime'] = runtime
# results['topics'] = topic_num
# results

In [21]:
##########################################################################################################################################

In [22]:
duration = 2  
sampling_rate = 44100
frequency = 440.0 
t = np.linspace(0, duration, int(sampling_rate * duration), False)
wave = 0.5 * np.sin(2 * np.pi * frequency * t)

audio = ipd.Audio(wave, rate=sampling_rate, autoplay=True)
display(audio)